In [1]:
import sys
sys.path.append("..") # Adds the module to path

# DeepTrack.properties

This notebook introduces DeepTrack.properties.

## What is a property?

Each feature (instance of the class Feature) can have several properties (instances of the class Property).
A propety has a value accessible through the `current_value` field, whose data type is not genrally restricted. 
This value is updated through a sampling rule (method `update()`), which is passed to the class constructor on initialization. 

## What is a sampling rule?

The sampling rule determines how the value of a property is updated upon calling `update()`.
A sampling rule is defined when an instance of the class Property is created and can be of any type. 
When calling `update()`, the value of the property is updated according to first the following that applies:
    
1.  If the sampling rule has a method `sample()`, call `sample()` and return the output.
2.  If the sampling rule is a dictionary, create an exact copy of the dictionary and substitute
    any value that has a `sample()` method with the result of the call to this method.
3.  If the sampling rule is either a ``list`` or a 1-dimensional ``ndarray``, extract one element randomly.
4.  If the sampling rule is an iterable, return the next value.
5.  If the sampling rule is callable, call it with no arguments and return the result.
6.  If none of the above apply, return the sampling rule itself.

In [2]:
import numpy as np
from DeepTrack.properties import Property

### 1 - Property with a constant value

The simplest example of a property is one that does not change during an update call.
This is commonly either a number or a tuple, but can be any data type that will be evaluated by case 6.
If you want to have a constant property with a value that would be evaluated by cases 1-5 (e.g., a list or a function), you can  wrap it as the output of a lambda function.

In [3]:
# NUMBER
print("NUMBER")

P = Property(1)
print("The current value of the property is", P.current_value)
P.update()
print("The current value of the property is", P.current_value) # Numbers are not changed after a update() call

# TUPLE
print("\nTUPLE")

P = Property((1,[2,3],None))
print("The current value of the property is", P.current_value)
P.update()
print("The current value of the property is", P.current_value) # Tuples are not changed after a update() call

# WRAPPED LIST
print("\nWRAPPED LIST")

P = Property(lambda: [1,2,3])
print("The current value of the property is", P.current_value)
P.update()
print("The current value of the property is", P.current_value) # Tuples are not changed after a update() call

NUMBER
The current value of the property is 1
The current value of the property is 1

TUPLE
The current value of the property is (1, [2, 3], None)
The current value of the property is (1, [2, 3], None)

WRAPPED LIST
The current value of the property is [1, 2, 3]
The current value of the property is [1, 2, 3]


### 2 - Property with a discrete random value 

Discrete randomness can be achieved by either a list, a 1-dimensional ndarray (case 3) or a function (case 5). For lists and ndarrays, the output is a single element of the list chosen uniformly random. For non-uniform sampling, either use lists with repeated elements, or a function. 

In [4]:
# LIST
print("LIST")

P = Property([1, 2, 3, 4])
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

# NDARRAY
print("\nNDARRAY")

P = Property(np.linspace(0,1,5))
for _ in range(5): 
    P.update()
    print("The current value of the property is", P.current_value)
    
# FUNCTION
print("\nFUNCTION")

P = Property(lambda: 1 if np.random.rand() > 0.9 else 0)
for _ in range(5): 
    P.update()
    print("The current value of the property is", P.current_value)

LIST
The current value of the property is 4
The current value of the property is 2
The current value of the property is 3
The current value of the property is 2
The current value of the property is 3

NDARRAY
The current value of the property is 0.0
The current value of the property is 0.75
The current value of the property is 0.75
The current value of the property is 0.75
The current value of the property is 1.0

FUNCTION
The current value of the property is 0
The current value of the property is 0
The current value of the property is 0
The current value of the property is 0
The current value of the property is 0


### 3 - Property with a continuous random value 

Continuous randomness is typically achieved by passing a function that returns a coninuously random value. This function should take no input, as noted in case 5. To use a function that needs arguments, wrap it in a function that calls it with the correct arguments.

In [5]:
# FUNCTION WITH NO INPUT
print("FUNCTION WITH NO INPUT")
P = Property(np.random.rand)
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

# WRAPPED FUNCTION
print("\nWRAPPED FUNCTION")
P = Property(lambda: np.random.normal(1, 5))
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

FUNCTION WITH NO INPUT
The current value of the property is 0.13308086453553136
The current value of the property is 0.8316488979902202
The current value of the property is 0.4313000361483551
The current value of the property is 0.435409450657286
The current value of the property is 0.17479289484872595

WRAPPED FUNCTION
The current value of the property is 6.512565198031771
The current value of the property is 6.036221384384144
The current value of the property is 9.357107181640213
The current value of the property is 1.4032090465605718
The current value of the property is 2.667482930453531


### 4 - Property with a deterministically changing value

Deterministically changing properties can be achieved using either an iterator (case 4) or a function. For the output of a function to change deterministically between calls it should reference some variable outside its definition.  


In [6]:
# ITERATOR
print("ITERATOR")
P = Property(iter([1,2,3,4,5]))
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

# FUNCTION
print("\nFUNCTION")
fibbonacci = [1,1]
def fibbonacci_sequence():
    fibbonacci.append(fibbonacci[-2] + fibbonacci[-1])
    return fibbonacci

P = Property(fibbonacci_sequence)
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

ITERATOR
The current value of the property is 1
The current value of the property is 2
The current value of the property is 3
The current value of the property is 4
The current value of the property is 5

FUNCTION
The current value of the property is [1, 1, 2]
The current value of the property is [1, 1, 2, 3]
The current value of the property is [1, 1, 2, 3, 5]
The current value of the property is [1, 1, 2, 3, 5, 8]
The current value of the property is [1, 1, 2, 3, 5, 8, 13]


## What is a PropertyDict?

The second class contained in the DeepTrack.properties module is called PropertyDict. This is a dictionary of properties with a few utility methods defined to make managing collections of properties easier. These include

* `current_value_dict()`, which creates and returns a dictionary with the current value of all properties in the PropertyDict.
* `update()`, which calls the method `update()` on all properties in the PropertyDict.
* `sample()`, which calls the method `sample()` on all properties in the PropertyDict, and creates and returns a dictionary from the output.


In [10]:
from DeepTrack.properties import PropertyDict, Property

property_dict = PropertyDict(
    foo=Property(1),
    bar=Property([1,2,3]),
    baz=Property(np.random.rand)
)

for _ in range(5):
    property_dict.update()
    print("The current value of the property dict is", property_dict.current_value_dict())

The current value of the property dict is {'foo': 1, 'bar': 1, 'baz': 0.7793750185388738}
The current value of the property dict is {'foo': 1, 'bar': 2, 'baz': 0.7924147341286129}
The current value of the property dict is {'foo': 1, 'bar': 1, 'baz': 0.9611409111716739}
The current value of the property dict is {'foo': 1, 'bar': 3, 'baz': 0.776662008561869}
The current value of the property dict is {'foo': 1, 'bar': 2, 'baz': 0.3308330896736138}
